In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input/alicedataset"))

# Any results you write to the current directory are saved as output.

['alice.txt']


In [15]:
import numpy as np
np.random.seed(13)

from keras.models import Sequential, Model
from keras.layers import Embedding, Reshape, Activation, Input
from keras.layers.merge import Dot
from keras.utils import np_utils
from keras.utils.data_utils import get_file
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import skipgrams
import numpy as np
np.random.seed(13)
from keras.models import Sequential
from keras.layers import Dense, Embedding, Reshape
from IPython.display import SVG
from keras.utils import np_utils
from keras.utils.data_utils import get_file
from keras.preprocessing.text import Tokenizer
# from keras.utils.visualize_util import model_to_dot, plot
# from gensim.models.doc2vec import Word2Vec

import gensim

In [19]:
path = '../input/alicedataset/alice.txt'
corpus = open(path).readlines()
corpus = [sentence for sentence in corpus if sentence.count(' ') >= 2]
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
V = len(tokenizer.word_index) + 1
V

3387

In [20]:
# import gensim.downloader as api

In [21]:
# path=api.load("fake-news")

In [22]:
# print(path)
# for i in path:
#     print(i)

In [23]:
dim_embedddings = 128

# inputs
w_inputs = Input(shape=(1, ), dtype='int32')
w = Embedding(V, dim_embedddings)(w_inputs)

# context
c_inputs = Input(shape=(1, ), dtype='int32')
c  = Embedding(V, dim_embedddings)(c_inputs)
o = Dot(axes=2)([w, c])
o = Reshape((1,), input_shape=(1, 1))(o)
o = Activation('sigmoid')(o)

SkipGram = Model(inputs=[w_inputs, c_inputs], outputs=o)
SkipGram.summary()
SkipGram.compile(loss='binary_crossentropy', optimizer='adam')



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, 1, 128)       433536      input_5[0][0]                    
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, 1, 128)       433536      input_6[0][0]                    
__________________________________________________________________________________________________
dot_3 (Dot

In [24]:
for _ in range(5):
    loss = 0.
    for i, doc in enumerate(tokenizer.texts_to_sequences(corpus)):
        data, labels = skipgrams(sequence=doc, vocabulary_size=V, window_size=5, negative_samples=5.)
        x = [np.array(x) for x in zip(*data)]
        y = np.array(labels, dtype=np.int32)
        if x:
            loss += SkipGram.train_on_batch(x, y)

    print(loss)

Instructions for updating:
Use tf.cast instead.
1081.5277689397335
744.5550323724747
691.9281262457371
662.1209198981524
634.0469603091478


In [33]:
d={}
# f = open('vectors.txt' ,'w')
# f.write('{} {}\n'.format(V-1, dim_embedddings))
vectors = SkipGram.get_weights()[0]
for word, i in tokenizer.word_index.items():
    d[word]=list(vectors[i, :])
# f.close()

In [69]:
print(d['city'])

[-0.038376633, 0.009719515, -0.0889027, -0.083733305, -0.0023773785, -0.033958938, -0.0529249, 0.030297158, 0.010152886, 0.061350428, 0.050730832, -0.046394154, 0.07699522, -0.055268608, -0.060266633, 0.07478753, 0.07128036, -0.0769707, 0.050335888, 0.05958583, 0.07972122, -0.018003779, -0.043756507, 0.050357487, 0.03604521, -0.046744127, 0.023621745, -0.022285206, 0.03634491, -0.031569608, -0.029099608, -0.00028811742, -0.022062441, 0.03211879, -0.028555196, 0.046905283, -0.059432924, -0.08504348, -0.009541368, 0.019290762, 0.022488536, 0.08994357, 0.053111628, -0.048452567, -0.019081004, 0.08726421, 0.091808, 0.034795526, -0.010350686, -0.066471994, -0.0062665585, 0.03392686, -0.031215921, 0.01466467, 0.016712781, -0.077040836, 0.004431684, 0.048801024, -0.056542218, -0.047240105, 0.011561346, -0.08643786, -0.0779489, -0.03215812, -0.080942474, -0.08169672, -0.072708584, -0.086208746, 0.077827804, 0.060280856, 0.0068738908, 0.048275862, -0.032527406, -0.004592836, 0.05459291, -0.0116

In [43]:
def avg_sentence_vector(words, model, num_features):
    featureVec = np.zeros((num_features,), dtype="float32")
    nwords = 0
    for word in words:
        if word in model.keys():
            featureVec = np.add(featureVec, model[word])

    return featureVec

In [70]:
sentence1 = ['i','will','be','eating','tea']
sentence2 = ['i','will','be','drinking','coffee']

In [71]:
feature1 = avg_sentence_vector(sentence1,d,128)
feature2 = avg_sentence_vector(sentence2,d,128)

In [72]:
from scipy.spatial.distance import cosine
print(cosine(feature1,feature2))

0.032353103160858154


In [73]:
sentence3 = ['I','am','walking','to','America']
sentence4 = ['I','am','going','to','Bharat']

In [74]:
feature3 = avg_sentence_vector(sentence3,d,128)
feature4 = avg_sentence_vector(sentence4,d,128)

In [75]:
print(cosine(feature3,feature4))

0.05034977197647095


In [68]:
# import cv2